In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

### a.모델 재정의

In [3]:
def model(t_u, w2, w1, b):
    return w2 * t_u **2 + w1 * t_u + b

### b. 모델 변경과 무관한 곳은 loss_fn이다.

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [6]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

In [11]:
if params.grad is not None:
    params.grad.zero_()

In [12]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

if params.grad is not None:
    params.grad.zero_()

In [13]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [17]:
training_loop(
    n_epochs=5000,
    learning_rate=1e-4,
    params=params,
    t_u=t_un,
    t_c=t_c)


Epoch 500, Loss 3.588712
Epoch 1000, Loss 3.583810
Epoch 1500, Loss 3.579174
Epoch 2000, Loss 3.574729
Epoch 2500, Loss 3.570423
Epoch 3000, Loss 3.566219
Epoch 3500, Loss 3.562092
Epoch 4000, Loss 3.558018
Epoch 4500, Loss 3.553990
Epoch 5000, Loss 3.549996


tensor([ 0.6257, -1.3133, -0.9631], requires_grad=True)

### c. 훈련 후 손실이 증가한 것으로 확인할 수 있다.